In [3]:
import pandas as pd

# Carregar o dataset
df = pd.read_csv("C:/github/Rio-Housing-Scrapper/Data(29-09_2023)/df_cleaned_final.csv")

# Exibir as primeiras linhas do dataframe
df.head()

,Property Type,Address,Region,Rent,Total Rent,Area,Rooms,Bathrooms,Garage,Furnished
0,Apartamento,Rua Marechal Bittencourt,Riachuelo,700,1368,55,2,1,1.0,0
1,Apartamento,Rua Tonelero,Copacabana,4200,6012,140,5,2,1.0,1
2,Apartamento,Rua Correa Dutra,Flamengo,2450,3198,53,1,1,0.0,0
3,Apartamento,Rua Theodor Herzl,Botafogo,3300,4075,61,2,1,0.0,0
4,Apartamento,Rua Hilário de Gouvêia,Copacabana,1750,2649,27,1,1,0.0,0


In [ ]:
from sklearn.model_selection import train_test_split

# Excluir as colunas "Rent" e "Address"
df = df.drop(columns=["Rent", "Address"])

# Definir variáveis independentes (features) e a variável dependente (target)
X = df.drop("Total Rent", axis=1)
y = df["Total Rent"]

# Dividir os dados em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Exibir o tamanho dos conjuntos de treino e teste
X_train.shape, X_test.shape, y_train.shape, y_test.shape
